In [ ]:
import numpy as np
from sympy import symbols
import math

# Define a class for nodes
class Node:
    def __init__(self, support_type, coordinates, degrees_of_freedom, reaction_forces_exist):
        self.support_type = support_type
        # self.external_forces_moment = external_forces_moment
        self.coordinates = coordinates
        self.degrees_of_freedom = degrees_of_freedom
        self.reaction_forces_exist = reaction_forces_exist
        # self.settlement = settlement

# Define a class for members
class Member:
    def __init__(self, node1_id, node2_id, cross_section_area, modulus_of_elasticity, moment_of_inertia, length, angle, loads_info):
        self.node1_id = node1_id
        self.node2_id = node2_id
        self.cross_section_area = cross_section_area
        self.modulus_of_elasticity = modulus_of_elasticity
        self.moment_of_inertia = moment_of_inertia
        self.length = length
        self.angle = angle
        self.loads_info = loads_info

# Function to determine degrees of freedom based on support type
def determine_degrees_of_freedom(support_type):
    u, v, theta = 0, 0, 0
    if support_type == 'fixed':
        u, v, theta = 0, 0, 0
    elif support_type == 'pinned':
        theta = 1
    elif support_type == 'hroller':
        u, theta = 1, 1
    elif support_type == 'vroller':
        v, theta = 1, 1
    elif support_type == 'free':
        u, v, theta = 1, 1, 1
    return u, v, theta

# Function to determine existence of external forces and moments based on support type
def determine_unknown_forces(support_type):
    H_exists, V_exists, M_exists = False, False, False

    if support_type == 'fixed':
        H_exists, V_exists, M_exists = True, True, True
    elif support_type == 'pinned':
        H_exists, V_exists = True, True
    elif support_type == 'hroller':
        V_exists = True
    elif support_type == 'vroller':
        H_exists = True
    elif support_type == 'free':
        H_exists, V_exists, M_exists = False, False, False

    return H_exists, V_exists, M_exists

# Function to take input for node details
def take_node_data(num_nodes):
    nodes = {}
    for i in range(num_nodes):
        print(f"\nEnter details for Node {i+1}:")
        support_type = input("Type of Support (fixed, pinned, hroller, vroller, free): ").strip().lower()
        # ext_input = input("External Forces and Moment (H, V, M separated by space, if none enter 0 0 0): ").strip().split()
        # ext_input = [float(f) for f in ext_input]
        # settlement_input = input("Settlement at Node (u_s v_s, theta_s separated by space, if none enter 0 0 0): ").strip().split()
        # settlement_input = [float(s) for s in settlement_input]
        coordinates = input("Coordinates of Node (x, y separated by space): ").strip().split()
        coordinates = [float(coord) for coord in coordinates]
        degrees_of_freedom = determine_degrees_of_freedom(support_type)
        Hu, Vu, Mu = determine_unknown_forces(support_type)
        node = Node(support_type, coordinates, degrees_of_freedom, {'H': Hu, 'V': Vu, 'M': Mu})
        nodes[i+1] = node
    return nodes

# Function to take input for member details
def take_member_data(num_members, nodes):
    members = {}
    for i in range(num_members):
        print(f"\nEnter details for Member {i+1}:")
        node1_id = int(input("Enter ID of first node: "))
        node2_id = int(input("Enter ID of second node: "))
        cross_section_area = float(input("Enter cross-section area of member: "))
        modulus_of_elasticity = float(input("Enter modulus of elasticity of member: "))
        moment_of_inertia = float(input("Enter moment of inertia of member: "))
        num_loads = int(input("Enter the number of loads on this member: "))
        loads_info = []
        for j in range(num_loads):
            load_type = input(f"Enter Load Type {j+1} (point_load, uniform_distributed_load, uniform_varying_load, trapezoidal_load): ").strip().lower()
            load_value = input(f"Enter Load Value {j+1} (separated by space, if none enter 0): ").strip().split()
            load_value = [float(f) for f in load_value]
            # load_distance = input(f"Enter Distance of Load {j+1} from Node 1 and Node 2 (separated by space as a b c, if none enter 0 0 0): ").strip().split()
            # load_distance = [float(f) for f in load_distance]
            load_distance = input(f"Enter Distance of Load {j+1} from Node 1 and Node 2 (separated by space as a b, if none enter 0 0): ").strip().split()
            load_distance = [float(f) for f in load_distance[:2]]  # Take only the first two values
            loads_info.append((load_type, load_value, load_distance))
        length = ((nodes[node2_id].coordinates[0] - nodes[node1_id].coordinates[0])**2 + (nodes[node2_id].coordinates[1] - nodes[node1_id].coordinates[1])**2) ** 0.5
        angle = math.atan2(nodes[node2_id].coordinates[1] - nodes[node1_id].coordinates[1], nodes[node2_id].coordinates[0] - nodes[node1_id].coordinates[0]) * 180 / math.pi
        member = Member(node1_id, node2_id, cross_section_area, modulus_of_elasticity, moment_of_inertia, length, angle, loads_info)
        members[i+1] = member
    return members

# Function to calculate member lengths and angles
def calculate_member_length(n_val, m_val):
    L = []  # List to store member lengths
    th = []  # List to store member angles

    for i in m_val.keys():
        node1 = m_val[i].node1_id
        node2 = m_val[i].node2_id
        x1, y1 = n_val[node1].coordinates
        x2, y2 = n_val[node2].coordinates
        length = ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
        angle = math.atan2(y2 - y1, x2 - x1) * 180 / math.pi
        L.append(length)
        th.append(angle)

    return L, th

# Function to calculate local stiffness matrix
def l_stiffness_matrix(A, E, L, I):
    k_local = np.zeros((6, 6))
    k_local[0, 0] = A * E / L
    k_local[0, 3] = -A * E / L
    k_local[1, 1] = 12 * E * I / (L ** 3)
    k_local[1, 2] = 6 * E * I / (L ** 2)
    k_local[1, 4] = -12 * E * I / (L ** 3)
    k_local[1, 5] = 6 * E * I / (L ** 2)
    k_local[2, 1] = 6 * E * I / (L ** 2)
    k_local[2, 2] = 4 * E * I / L
    k_local[2, 4] = -6 * E * I / (L ** 2)
    k_local[2, 5] = 2 * E * I / L
    k_local[3, 3] = A * E / L
    k_local[3, 0] = -A * E / L
    k_local[4, 1] = -12 * E * I / (L ** 3)
    k_local[4, 2] = -6 * E * I / (L ** 2)
    k_local[4, 4] = 12 * E * I / (L ** 3)
    k_local[4, 5] = -6 * E * I / (L ** 2)
    k_local[5, 1] = 6 * E * I / (L ** 2)
    k_local[5, 2] = 2 * E * I / L
    k_local[5, 4] = -6 * E * I / (L ** 2)
    k_local[5, 5] = 4 * E * I / L
    return k_local

# Function to perform transformation to global coordinate system
def transformation(theta, transpose=False):
    c = np.cos(np.radians(theta))
    s = np.sin(np.radians(theta))
    trans_matrix = np.array([
        [c, -s, 0, 0, 0, 0],
        [s, c, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, c, -s, 0],
        [0, 0, 0, s, c, 0],
        [0, 0, 0, 0, 0, 1]
    ])
    if transpose:
        trans_matrix = np.transpose(trans_matrix)
    return trans_matrix

# Function to calculate global stiffness matrix
def calculate_global_stiffness_matrix(member_data, nodes):
    num_elements = len(member_data)
    k_local_list = [l_stiffness_matrix(member_data[i].cross_section_area, member_data[i].modulus_of_elasticity, member_data[i].length, member_data[i].moment_of_inertia) for i in range(1, num_elements + 1)]
    num_nodes = num_elements + 1
    num_dofs = 3
    K_global = np.zeros((num_nodes * num_dofs, num_nodes * num_dofs))
    t_matrix_list = [transformation(member_data[i].angle) for i in range(1, num_elements + 1)]
    for i in range(num_elements):
        k_local_transformed = np.dot(np.dot(t_matrix_list[i], k_local_list[i]), np.transpose(t_matrix_list[i]))
        start_index = (i * num_dofs)
        end_index = ((i + 2) * num_dofs)
        for m in range(num_dofs * 2):
            for n in range(num_dofs * 2):
                K_global[start_index + m, start_index + n] += k_local_transformed[m, n]

    print("Global Stiffness Matrix:")
    print(K_global)
    return K_global

def calculate_displacement_matrix(nodes):
    disp_values = []
    for node_id, node in nodes.items():
        u = symbols('u_' + str(node_id)) if node.degrees_of_freedom[0] else 0
        v = symbols('v_' + str(node_id)) if node.degrees_of_freedom[1] else 0
        theta = symbols('theta_' + str(node_id)) if node.degrees_of_freedom[2] else 0
        disp_values.extend([u, v, theta])
    disp_matrix = np.array(disp_values).reshape(-1, 1)
    print("\nDisplacement Matrix:")
    print(disp_matrix)
    return disp_matrix


# Function to calculate external force matrix
def calculate_external_force_matrix(nodes, external_forces):
    external_force = []
    for node_id, node in nodes.items():
        H = external_forces[node_id][0]
        V = external_forces[node_id][1]
        M = external_forces[node_id][2]
        external_force.extend([H, V, M])
    external_force_matrix = np.array(external_force).reshape(-1, 1)
    print("\nExternal Force Matrix:")
    print(np.array(external_force_matrix))
    return external_force_matrix

#Settlement
def calculate_settlement_matrix(nodes, settlements):
    settlement = []
    for node_id, node in nodes.items():
        u_s = settlements[node_id][0]
        v_s = settlements[node_id][1]
        theta_s = settlements[node_id][2]
        settlement.extend([u_s, v_s, theta_s])
    settlement_matrix = np.array(settlement).reshape(-1, 1)
    print("\nSettlement_matrix:")
    print(np.array(settlement_matrix))
    return settlement_matrix

# Function to calculate unknown force matrix
def calculate_unknown_force_matrix(nodes):
    unknown_force = []
    for node_id, node in nodes.items():
        support_type = node.support_type
        H_exists, V_exists, M_exists = determine_unknown_forces(support_type)

        Hu = symbols('Hu_' + str(node_id)) if H_exists else 0
        Vu = symbols('Vu_' + str(node_id)) if V_exists else 0
        Mu = symbols('Mu_' + str(node_id)) if M_exists else 0

        unknown_force.extend([Hu, Vu, Mu])

    unknown_force_matrix = np.array(unknown_force).reshape(-1, 1)
    print("\nUnknown Force Matrix:")
    print(np.array(unknown_force_matrix))
    return unknown_force_matrix


In [ ]:
num_nodes = int(input("Enter the number of nodes: "))
nodes_data = take_node_data(num_nodes)

external_forces = {}  # Initialize external forces dictionary
for i in range(num_nodes):
    H, V, M = input(f"Enter external forces (H, V, M) for Node {i+1} separated by space: ").split()
    external_forces[i+1] = (float(H), float(V), float(M))  # Store external forces for each node
settlements = {}  # Initialize external forces dictionary
for i in range(num_nodes):
    u_s, v_s, theta_s = input(f"Enter settlements (u_s, v_s, theta_s) for Node {i+1} separated by space: ").split()
    settlements[i+1] = (float(u_s), float(v_s), float(theta_s))  # Store external forces for each node

num_members = int(input("Enter the number of members: "))
member_data = take_member_data(num_members, nodes_data)

Enter the number of nodes: 3

Enter details for Node 1:
Type of Support (fixed, pinned, hroller, vroller, free): fixed
Coordinates of Node (x, y separated by space): 0 0

Enter details for Node 2:
Type of Support (fixed, pinned, hroller, vroller, free): free
Coordinates of Node (x, y separated by space): 0 10

Enter details for Node 3:
Type of Support (fixed, pinned, hroller, vroller, free): fixed
Coordinates of Node (x, y separated by space): 10 10
Enter external forces (H, V, M) for Node 1 separated by space: 0 0 0
Enter external forces (H, V, M) for Node 2 separated by space: 100 0 0
Enter external forces (H, V, M) for Node 3 separated by space: 0 0 0
Enter settlements (u_s, v_s, theta_s) for Node 1 separated by space: 0 0 0
Enter settlements (u_s, v_s, theta_s) for Node 2 separated by space: 0 0 0
Enter settlements (u_s, v_s, theta_s) for Node 3 separated by space: 0 0 0
Enter the number of members: 2

Enter details for Member 1:
Enter ID of first node: 1
Enter ID of second node: 2

In [ ]:
L_val, th_val = calculate_member_length(nodes_data, member_data)
global_stiffness_matrix = calculate_global_stiffness_matrix(member_data, nodes_data)
# global_fixed_end_forces = calculate_global_fixed_end_forces(member_data)
displacement_matrix = calculate_displacement_matrix(nodes_data)
external_force_matrix = calculate_external_force_matrix(nodes_data, external_forces)
unknown_force_matrix = calculate_unknown_force_matrix(nodes_data)
settlement_matrix = calculate_settlement_matrix(nodes_data, settlements)
s = settlement_matrix + displacement_matrix
print(L_val)
print(th_val)

Global Stiffness Matrix:
[[ 4.37472000e-01  1.07153916e-12 -2.18736000e+00 -4.37472000e-01
  -1.07153916e-12 -2.18736000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.07153916e-12  1.75000000e+04  1.33937171e-16 -1.07153916e-12
  -1.75000000e+04  1.33937171e-16  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-2.18736000e+00  1.33937171e-16  1.45824000e+01  2.18736000e+00
  -1.33937171e-16  7.29120000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-4.37472000e-01 -1.07153916e-12  2.18736000e+00  1.75004375e+04
   1.07153916e-12  2.18736000e+00 -1.75000000e+04  0.00000000e+00
   0.00000000e+00]
 [-1.07153916e-12 -1.75000000e+04 -1.33937171e-16  1.07153916e-12
   1.75004375e+04  2.18736000e+00  0.00000000e+00 -4.37472000e-01
   2.18736000e+00]
 [-2.18736000e+00  1.33937171e-16  7.29120000e+00  2.18736000e+00
   2.18736000e+00  2.91648000e+01  0.00000000e+00 -2.18736000e+00
   7.29120000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.75000000e+04
   

In [ ]:
def calculate_global_fixed_end_forces(member_data):
    num_elements = len(member_data)
    no_node = num_elements + 1
    global_fixed_end_forces_matrix = np.zeros((3 * no_node, 1))

    for i, data in member_data.items():
        for load_info in data.loads_info:
            load_type, load_value, load_distance = load_info
            length = data.length
            angle = data.angle
            print(angle)

            F1, F2, M1, M2 = 0, 0, 0, 0

            if load_type == 'point_load':
                F1, F2, M1, M2 = point_load_fixed_end(length, load_value[0], load_distance[0], load_distance[1])
            elif load_type == 'uniform_distributed_load':
                F1, F2, M1, M2 = uniform_distributed_load_fixed_end(length, load_value[0],load_distance[0], load_distance[1])
            elif load_type == 'uniform_varying_load':
                F1, F2, M1, M2 = uniform_varying_load_fixed_end(length, load_value[0], load_value[1],load_distance[0], load_distance[1])
            elif load_type == 'trapezoidal_load':
                F1, F2, M1, M2 = trapezoidal_load_fixed_end(length, load_value[0], load_value[1])

            local_fixed = np.zeros((6,1))

            # Assign values to the corresponding positions
            local_fixed[0] = 0
            local_fixed[1] = F1
            local_fixed[2] = M1
            local_fixed[3] = 0
            local_fixed[4] = F2
            local_fixed[5] = M2


# Anticlockwise positive(moment)
            node1_id = data.node1_id
            node2_id = data.node2_id
            t_matrix = [transformation(member_data[i].angle)]
            print(t_matrix)
            print(local_fixed)
            local_fixed_transformed = np.dot(t_matrix, local_fixed)
            print(local_fixed_transformed)
            local_fixed_t = local_fixed_transformed.reshape(6, 1)
            print(local_fixed_t)
            # Assemble local forces to global fixed end forces matrix
            global_fixed_end_forces_matrix[(node1_id - 1) * 3:(node1_id - 1) * 3 + 6] += local_fixed_t

    print("\nGlobal Fixed End Forces Matrix:")
    print(global_fixed_end_forces_matrix)
    return global_fixed_end_forces_matrix

# Function to calculate fixed end forces due to point load
def point_load_fixed_end(L, P, a, b):
    F1 = -(P * (b ** 2) * (L + (2 * a))) / (L ** 3)
    F2 = -(P * (a ** 2) * (L + (2 * b))) / (L ** 3)
    M1 = -(P * (b ** 2) * a) / L ** 2
    M2 = (P * (a ** 2) * b) / L ** 2
    return F1, F2, M1, M2


# Function to calculate fixed end forces due to uniform distributed load
def uniform_distributed_load_fixed_end(L, w, a, b):
    c = L-a-b
    d = b+(c/2)
    e = a+c
    F1 = -(w*c*((12*(d**2))-((8*(d**3))/(L)) + ((2*e*(c**2))/(L)) - ((c**3)/(L)) - (c**2)))/(4*(L**2))
    M1 = -(w*c*(((24*(d**3))/(L))-((6*e*(c**2))/(L))+((3*(c**3))/(L))+(4*(c**2))-(24*(d**2))))/(24*L)
    F2 = -(w*c)+((w*c*((12*(d**2))-((8*(d**3))/(L))+((2*e*(c**2))/(L))-((c**3)/(L))-(c**2)))/(4*(L**2)))
    M2 = (w*c*(((24*(d**3))/(L))-((6*e*(c**2))/(L))+((3*(c**3))/(L))+(4*(c**2))-(24*(d**2))))/(24*L)
    return F1, F2, M1, M2

# Function to calculate fixed end forces due to uniform varying load
def uniform_varying_load_fixed_end(L, w1, w2,s1,s3):
    s2 = L-s1-s3

    M2 = -((-1/60)*(w2)*(s2)*((3*(s2**3))+((15*(s2**2)*(s3)))+((10*(s1**2)*(s2)))+((30*(s1**2)*(s3)))+((10*(s2**2)*(s1)))+((40*(s2)*(s1)*(s3))))/((s1+s2+s3)**2)) + ((-1/60)*(w1)*(s2)*((2*(s2**3))+((5*(s2**2)*(s3)))+((20*(s1**2)*(s2)))+((30*(s1**2)*(s3)))+((10*(s2**2)*(s1)))+ ((20*(s2)*(s1)*(s3))))/((s1+s2+s3)**2))
    M1 = (((-1/60)*(w1)*(s2)*((3*(s2**3))+((15*(s2**2)*(s1)))+((10*(s3**2)*(s2)))+((30*(s3**2)*(s1)))+((10*(s2**2)*(s3)))+((40*(s2)*(s1)*(s3))))/((s1+s2+s3)**2)) + ((-1/60)*(w2)*(s2)*((2*(s2**3))+((5*(s2**2)*(s1)))+((20*(s3**2)*(s2)))+((30*(s3**2)*(s1)))+((10*(s2**2)*(s3)))+ ((20*(s2)*(s1)*(s3))))/((s1+s2+s3)**2)))
    F2 = -((1/20)*(w1)*(s2)*((3*(s2**3))+((5*(s2**2)*(s3)))+((10*(s1**3)*(1)))+((30*(s1**2)*(s2)))+((30*(s1**2)*(s3)))+((15*(s2**2)*(s1)))+((20*(s2)*(s1)*(s3))))/((s1+s2+s3)**3)) + ((1/20)*(w2)*(s2)*((7*(s2**3))+((15*(s2**2)*(s3)))+((10*(s1**3)*(1)))+((30*(s1**2)*(s2)))+((30*(s1**2)*(s3)))+((25*(s2**2)*(s1)))+((40*(s2)*(s1)*(s3))))/((s1+s2+s3)**3))
    F1 = -((1/20)*(w2)*(s2)*((3*(s2**3))+((5*(s2**2)*(s1)))+((10*(s3**3)*(1)))+((30*(s3**2)*(s2)))+((30*(s3**2)*(s1)))+((15*(s2**2)*(s3)))+((20*(s2)*(s3)*(s1))))/((s1+s2+s3)**3)) + ((1/20)*(w1)*(s2)*((7*(s2**3))+((15*(s2**2)*(s1)))+((10*(s3**3)*(1)))+((30*(s3**2)*(s2)))+((30*(s3**2)*(s1)))+((25*(s2**2)*(s3)))+((40*(s2)*(s1)*(s3))))/((s1+s2+s3)**3))
    # F1 = (w1 + 2 * w2) * L / 6
    # F2 = (2 * w1 + w2) * L / 6
    # M1 = w1 * L ** 2 / 12
    # M2 = w2 * L ** 2 / 12
    return F1, F2, M1, M2

# Function to calculate fixed end forces due to trapezoidal load
def trapezoidal_load_fixed_end(L, w1, w2):
    F1 = (w1 + w2) * L / 4
    F2 = (w1 + w2) * L / 4
    M1 = w1 * L ** 2 / 12
    M2 = w2 * L ** 2 / 12
    return F1, F2, M1, M2
global_fixed_end_forces = calculate_global_fixed_end_forces(member_data)

0.0
[array([[ 1., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1., -0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.]])]
[[   0.]
 [ -50.]
 [-125.]
 [   0.]
 [ -50.]
 [ 125.]]
[[[   0.]
  [ -50.]
  [-125.]
  [   0.]
  [ -50.]
  [ 125.]]]
[[   0.]
 [ -50.]
 [-125.]
 [   0.]
 [ -50.]
 [ 125.]]

Global Fixed End Forces Matrix:
[[   0.]
 [   0.]
 [   0.]
 [   0.]
 [ -50.]
 [-125.]
 [   0.]
 [ -50.]
 [ 125.]]


In [ ]:
result = np.dot(global_stiffness_matrix, s)
result_with_forces = result + global_fixed_end_forces - external_force_matrix - unknown_force_matrix
print("Resultant matrix:")
print(result_with_forces)

Resultant matrix:
[[-Hu_1 - 2.18736*theta_2 - 0.437472*u_2 - 1.07153916181971e-12*v_2]
 [-Vu_1 + 1.33937171129148e-16*theta_2 - 1.07153916181971e-12*u_2 - 17500.0*v_2]
 [-Mu_1 + 7.2912*theta_2 + 2.18736*u_2 - 1.33937171129148e-16*v_2]
 [2.18736*theta_2 + 17500.437472*u_2 + 1.07153916181971e-12*v_2 - 100.0]
 [2.18736*theta_2 + 1.07153916181971e-12*u_2 + 17500.437472*v_2 - 50.0]
 [29.1648*theta_2 + 2.18736*u_2 + 2.18736*v_2 - 125.0]
 [-Hu_3 - 17500.0*u_2]
 [-Vu_3 - 2.18736*theta_2 - 0.437472*v_2 - 50.0]
 [-Mu_3 + 7.2912*theta_2 + 2.18736*v_2 + 125.0]]


In [ ]:
import sympy as sp
equations = [i[0] for i in result_with_forces]
variables = set()
for eq in equations:
    variables.update(eq.free_symbols)
x= sp.symbols(' '.join(str(variable) for variable in variables))
solution = sp.solve(equations,x)
print("Solution:")
print(solution)

Solution:
{Hu_1: -9.37603507181426, Hu_3: -90.6239649281857, Mu_1: 31.2572259962556, Mu_3: 156.250976552482, Vu_1: -40.6252148169405, Vu_3: -59.3747851830595, theta_2: 4.28542609249936, u_2: 0.00517851228161061, v_2: 0.00232144084668232}


In [ ]:
from tabulate import tabulate
# Convert the solution dictionary to a list of lists
solution_table = [["Variable", "Value"]]
solution_table.extend([[key, value] for key, value in solution.items()])

# Print the solution table
print(tabulate(solution_table, headers="firstrow"))


Variable           Value
----------  ------------
Hu_1         -9.37604
Hu_3        -90.624
Mu_1         31.2572
Mu_3        156.251
Vu_1        -40.6252
Vu_3        -59.3748
theta_2       4.28543
u_2           0.00517851
v_2           0.00232144
